# Iterative pruning pipeline
Model: Multi Layer Perceptron

*Pruning functions as class methods*

In [1]:
EXPERIMENT_NAME = 'mlp-global-random-unstruct'
ITERATIONS = 3 #should be 10 for final experiment


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import json
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
from tqdm import tqdm
import foolbox as fb
import random


tf.compat.v1.enable_eager_execution()
tf.keras.backend.clear_session()  # For easy reset of notebook state.

# Prune, Train Attack Pipeline

In [64]:

pgd_success_rates = []
cw2_success_rates = []

all_accuracies = []
for j in tqdm(range(ITERATIONS)):
    model_for_glob_pruning = initialize_base_model(j, save_weights=True)

    accuracies = []
    pgd_success_rate = []
    cw2_success_rate = []
    compression_rates = [1, 2, 4, 8, 16, 32, 64]
    pruning_ratios = [1-1/x for x in compression_rates]
    for index, pruning_ratio in tqdm(enumerate(pruning_ratios)):
        model.load_weights(f'./saved-weights/{EXPERIMENT_NAME}-{j}')

        #iteratively prune and train (only to convergence if the final stage of pruning is reached)
        for i in range(index + 1):
            if i != index:
                #glocbal pruning
                model.prune_random_local_unstruct(pruning_ratio)
                #model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                #              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                #              metrics=['accuracy'],
                #             )
                             
                #finetuning
                model = train_model(model, to_convergence=False)
            if i == index:
                print('final pruning and eval')
                model.prune_random_local_unstruct(pruning_ratio)
                #model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                #              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                #              metrics=['accuracy'],
                #             )
                #finetuning
                model = train_model(model, to_convergence=True)
                accuracies.append(model.evaluate(x_test, y_test, verbose=0))
                pgd_success_rate.append(pgd_attack(model))
                #cw2_success_rate.append(cw2_attack(model))
    all_accuracies.append(accuracies)
    pgd_success_rates.append(pgd_success_rate)
    cw2_success_rates.append(cw2_success_rate)

    
pd.DataFrame(all_accuracies).to_csv(f'saved-results/{EXPERIMENT_NAME}-accuracies.csv',index=False)
with open(f'saved-results/{EXPERIMENT_NAME}-accuracies.json', 'w') as f:
    json.dump(all_accuracies, f)
    
pd.DataFrame(pgd_success_rates).to_csv(f'saved-results/{EXPERIMENT_NAME}-pgd-success.csv',index=False)
with open(f'saved-results/{EXPERIMENT_NAME}-pgd-success.json', 'w') as f:
    json.dump(pgd_success_rates, f)
    
pd.DataFrame(cw2_success_rates).to_csv(f'saved-results/{EXPERIMENT_NAME}-cw2-success.csv',index=False)
with open(f'saved-results/{EXPERIMENT_NAME}-cw2-success.json', 'w') as f:
    json.dump(cw2_success_rates, f)

  0%|          | 0/3 [00:00<?, ?it/s]

938/938 [==============================] - 4s 4ms/step - loss: 1.5769 - accuracy: 0.8973 - val_loss: 1.5198 - val_accuracy: 0.9438



0it [00:00, ?it/s]

final pruning and eval
Epoch 1/100
938/938 [==============================] - 3s 4ms/step - loss: 1.5115 - accuracy: 0.9524 - val_loss: 1.5059 - val_accuracy: 0.9569
Epoch 2/100
938/938 [==============================] - 3s 4ms/step - loss: 1.4983 - accuracy: 0.9647 - val_loss: 1.4955 - val_accuracy: 0.9671
Epoch 3/100
938/938 [==============================] - 3s 4ms/step - loss: 1.4911 - accuracy: 0.9717 - val_loss: 1.4966 - val_accuracy: 0.9656
Epoch 4/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4870 - accuracy: 0.9753 - val_loss: 1.4928 - val_accuracy: 0.9688
Epoch 5/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4841 - accuracy: 0.9777 - val_loss: 1.4898 - val_accuracy: 0.9716
Epoch 6/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4813 - accuracy: 0.9805 - val_loss: 1.4870 - val_accuracy: 0.9754
Epoch 7/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4798 - accuracy: 0.9818 - val_loss: 1.4876


1it [00:48, 48.23s/it]

...

Epoch 1/2
938/938 [==============================] - 4s 4ms/step - loss: 2.2777 - accuracy: 0.1282 - val_loss: 2.2158 - val_accuracy: 0.1902
Epoch 2/2
938/938 [==============================] - 4s 4ms/step - loss: 2.0528 - accuracy: 0.4368 - val_loss: 1.9187 - val_accuracy: 0.6340
Epoch 1/2
938/938 [==============================] - 4s 4ms/step - loss: 1.8593 - accuracy: 0.6729 - val_loss: 1.8190 - val_accuracy: 0.7057
Epoch 2/2
938/938 [==============================] - 4s 4ms/step - loss: 1.8122 - accuracy: 0.7111 - val_loss: 1.7904 - val_accuracy: 0.7209
Epoch 1/2
938/938 [==============================] - 4s 4ms/step - loss: 1.7461 - accuracy: 0.7670 - val_loss: 1.6972 - val_accuracy: 0.7930
Epoch 2/2
938/938 [==============================] - 4s 4ms/step - loss: 1.6934 - accuracy: 0.7982 - val_loss: 1.6821 - val_accuracy: 0.8043
Epoch 1/2
938/938 [==============================] - 4s 4ms/step - loss: 1.6823 - accuracy: 0.8049 - val_loss: 1.6747 - val_accuracy: 0.8082
Epoch 2/2
938


6it [12:47, 136.19s/it]

Epoch 1/2
938/938 [==============================] - 3s 4ms/step - loss: 2.3016 - accuracy: 0.1440 - val_loss: 2.2960 - val_accuracy: 0.1800
Epoch 2/2
938/938 [==============================] - 3s 4ms/step - loss: 2.2849 - accuracy: 0.1625 - val_loss: 2.2744 - val_accuracy: 0.1394
Epoch 1/2
938/938 [==============================] - 4s 5ms/step - loss: 2.2082 - accuracy: 0.2575 - val_loss: 2.1544 - val_accuracy: 0.3432
Epoch 2/2
938/938 [==============================] - 5s 5ms/step - loss: 2.1074 - accuracy: 0.3809 - val_loss: 2.0696 - val_accuracy: 0.4071
Epoch 1/2
938/938 [==============================] - 4s 4ms/step - loss: 2.0558 - accuracy: 0.4288 - val_loss: 2.0332 - val_accuracy: 0.4436
Epoch 2/2
938/938 [==============================] - 4s 4ms/step - loss: 2.0230 - accuracy: 0.4499 - val_loss: 2.0147 - val_accuracy: 0.4552
Epoch 1/2
938/938 [==============================] - 4s 4ms/step - loss: 2.0112 - accuracy: 0.4600 - val_loss: 2.0067 - val_accuracy: 0.4625
Epoch 2/2
938

938/938 [==============================] - 4s 4ms/step - loss: 1.9740 - accuracy: 0.5001 - val_loss: 1.9745 - val_accuracy: 0.4984
Epoch 47/100
938/938 [==============================] - 4s 4ms/step - loss: 1.9739 - accuracy: 0.5002 - val_loss: 1.9747 - val_accuracy: 0.4994
Epoch 48/100
938/938 [==============================] - 4s 4ms/step - loss: 1.9738 - accuracy: 0.5008 - val_loss: 1.9743 - val_accuracy: 0.4989
Epoch 49/100
938/938 [==============================] - 5s 5ms/step - loss: 1.9736 - accuracy: 0.5011 - val_loss: 1.9745 - val_accuracy: 0.4979
Epoch 50/100
938/938 [==============================] - 4s 4ms/step - loss: 1.9735 - accuracy: 0.5006 - val_loss: 1.9742 - val_accuracy: 0.5003
Epoch 51/100
938/938 [==============================] - 4s 5ms/step - loss: 1.9734 - accuracy: 0.5013 - val_loss: 1.9746 - val_accuracy: 0.4984
Epoch 52/100
938/938 [==============================] - 4s 4ms/step - loss: 1.9733 - accuracy: 0.5013 - val_loss: 1.9743 - val_accuracy: 0.4982
Epoch


7it [17:10, 147.21s/it]
100%|██████████| 3/3 [49:20<00:00, 986.69s/it]


# Helper Functions

In [61]:
def train_model(model, to_convergence=True):
    if to_convergence == True:
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=100,
            callbacks=[callback],
            validation_data=(x_test, y_test),
            )
    if to_convergence == False:
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=2,
            validation_data=(x_test, y_test),
            )
    return model


def pgd_attack(model_to_attack):
    fmodel = fb.models.TensorFlowModel(model_to_attack, bounds=(0,1))
    attack = fb.attacks.LinfProjectedGradientDescentAttack()
    adversarials = attack(
        fmodel,
        x,
        y,
        epsilons=[15/255]
    )
    return np.count_nonzero(adversarials[2])/len(y)

def cw2_attack(model_to_attack):
    fmodel = fb.models.TensorFlowModel(model_to_attack, bounds=(0,1))
    attack = fb.attacks.L2CarliniWagnerAttack()
    adversarials = attack(
        fmodel,
        x,
        y,
        epsilons=[.5]
    )
    return np.count_nonzero(adversarials[2])/len(y)

def initialize_base_model(index, save_weights=False):
    model = LeNet300_100()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                  metrics=['accuracy'],
                  experimental_run_tf_function=False
                 )

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    model.fit(x=x_train,
              y=y_train,
              batch_size=64,
              epochs=1,
              callbacks=[callback],
              validation_data=(x_test, y_test),
             )
    if save_weights == True:
        model.save_weights(f'./saved-weights/{EXPERIMENT_NAME}-{index}')
    return model
    

# Load Data

In [60]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

x = tf.convert_to_tensor(x_train[:500].reshape(500,28*28))
y = tf.convert_to_tensor([y_train[:500]])[0];

# Define Model

In [88]:
class CustomLayer(layers.Layer):

    def __init__(self, units=32, activation='relu'):
        super(CustomLayer, self).__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        #print(input_shape)
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                name='unpruned_weights')
        self.mask = self.add_weight(shape=(self.w.shape),
                                    initializer='ones',
                                    trainable=False,
                                   name='pruning_mask')

        
    def call(self, inputs):
        #self.mask_2 = tf.multiply(self.mask, self.mask_2)
        x = tf.multiply(self.w, self.mask)
        #print(self.pruned_w.eval())
        x = tf.matmul(inputs, x)
        
        if self.activation == 'relu':
            return tf.keras.activations.relu(x)
        if self.activation == 'softmax':
            return tf.keras.activations.softmax(x)
        raise ValueError('Activation function not implemented')

class LeNet300_100(tf.keras.Model):
    def __init__(self):
        super(LeNet300_100, self).__init__()
        self.dense1 = CustomLayer(300)
        self.dense2 = CustomLayer(100)
        self.dense3 = CustomLayer(10, activation='softmax')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)
    
    def prune_random_global_unstruct(self, ratio):
        weights = self.get_weights()
        layers_to_prune = [0, 2, 4]
        flat_weights = np.array([])
        flat_mask = np.array([])
        for x in layers_to_prune:
            flat_weights = np.concatenate([flat_weights, weights[x].flatten()])
            flat_mask = np.concatenate([flat_mask, weights[x+1].flatten()])
        no_of_weights_to_prune = int(ratio * len(flat_weights))
        # find unpruned weights
        non_zero_weights = np.nonzero(flat_mask)[0]
        
        # calculate the amount of weights to be pruned this round
        no_of_weights_to_prune_left = int(no_of_weights_to_prune - (len(flat_weights) - len(non_zero_weights)) )
        # shuffle all non-zero weights
        random.shuffle(non_zero_weights)
        # and take the indices of the first x weights where x is the number of weights to be pruned this round
        indices_to_delete = non_zero_weights[:no_of_weights_to_prune_left]
        for idx_to_delete in indices_to_delete:
            flat_mask[idx_to_delete] = 0
            flat_weights[idx_to_delete] = 0
            
        #reshape
        z = 0
        for x in layers_to_prune:
            weights[x] = flat_weights[z:z + np.prod(weights[x].shape)].reshape(weights[x].shape)
            weights[x + 1] = flat_mask[z:z + np.prod(weights[x].shape)].reshape(weights[x].shape)
            z = z + np.prod(weights[x].shape)
        self.set_weights(weights)
        return True
        
            
        
        
    def prune_random_local_unstruct(self, ratio):
        layers = self.get_weights()
        layers_to_prune = [0, 2, 4]
        for index, weights in enumerate(layers):
            if index in layers_to_prune:
                shape = weights.shape
                flat_weights = weights.flatten()
                flat_mask = layers[index+1].flatten()
                no_of_weights_to_prune = int(len(flat_weights)*ratio)
                # find unpruned weights
                non_zero_weights = np.nonzero(flat_mask)[0]
                # calculate the amount of weights to be pruned this round
                no_of_weights_to_prune_left = int(no_of_weights_to_prune - (len(flat_weights) - len(non_zero_weights)) )
                # shuffle all non-zero weights
                random.shuffle(non_zero_weights)
                # and take the indices of the first x weights where x is the number of weights to be pruned this round
                
                indices_to_delete = non_zero_weights[:no_of_weights_to_prune_left]
                for idx_to_delete in indices_to_delete:
                    flat_mask[idx_to_delete] = 0
                    flat_weights[idx_to_delete] = 0
                
                mask_reshaped = flat_mask.reshape(shape)
                weights_reshaped = flat_weights.reshape(shape)
                layers[index+1] = mask_reshaped
                layers[index] = weights_reshaped
        self.set_weights(layers)
        return 
        
    
    def prune_magnitude_global_unstruct(self,ratio):
                
        shape1 = self.dense1.w.shape
        shape2 = self.dense2.w.shape
        shape3 = self.dense3.w.shape

        flat_weights = np.append(self.dense1.w.numpy().flatten() ,self.dense2.w.numpy().flatten())
        flat_weights = np.append(flat_weights ,self.dense3.w.numpy().flatten())
        flat_mask = np.append(self.dense1.mask.numpy().flatten(), self.dense2.mask.numpy().flatten())
        flat_mask = np.append(flat_mask, self.dense3.mask.numpy().flatten())
        
        no_of_weights_to_prune = int(len(flat_weights)*ratio)
        indices_to_delete = np.abs(flat_weights).argsort()[:no_of_weights_to_prune]
        
        for idx_to_delete in indices_to_delete:
            flat_mask[idx_to_delete] = 0
            flat_weights[idx_to_delete] = 0
            
        w1 = flat_weights[:shape1[0]*shape1[1]].reshape(shape1)
        w2 = flat_weights[shape1[0]*shape1[1]:shape1[0]*shape1[1]+shape2[0]*shape2[1]].reshape(shape2)
        w3 = flat_weights[-shape3[0]*shape3[1]:].reshape(shape3)
        m1 = flat_mask[:shape1[0]*shape1[1]].reshape(shape1)
        m2 = flat_mask[shape1[0]*shape1[1]:shape1[0]*shape1[1]+shape2[0]*shape2[1]].reshape(shape2)
        m3 = flat_mask[-shape3[0]*shape3[1]:].reshape(shape3)
        self.set_weights([w1,m1,w2,m2,w3,m3])
        #print(weights)
        return
    
    def prune_magnitude_local_unstruct(self, ratio):
        layers = self.get_weights()
        layers_to_prune = [0, 2, 4]
        for index, weights in enumerate(layers):
            if index in layers_to_prune:
                shape = weights.shape
                flat_weights = weights.flatten()
                mask = layers[index+1].flatten()
                
                no_of_weights_to_prune = int(len(flat_weights)*ratio)
                indices_to_delete = np.abs(flat_weights).argsort()[:no_of_weights_to_prune]
                for idx_to_delete in indices_to_delete:
                    mask[idx_to_delete] = 0
                    flat_weights[idx_to_delete] = 0
                
                mask_reshaped = mask.reshape(shape)
                weights_reshaped = flat_weights.reshape(shape)
                layers[index+1] = mask_reshaped
                layers[index] = weights_reshaped
        self.set_weights(layers)
        return True

# Compile and Train Model

In [89]:
model = LeNet300_100()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
              metrics=['accuracy'],
              experimental_run_tf_function=False
             )

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(x=x_train,
          y=y_train,
          batch_size=128,
          epochs=1,
          callbacks=[callback],
          validation_data=(x_test, y_test),
         )

model.save('./saved-models/mini-pipeline-mlp-baseline-model')
model.save_weights('./saved-models/weights')

469/469 [==============================] - 3s 5ms/step - loss: 1.6067 - accuracy: 0.8786 - val_loss: 1.5292 - val_accuracy: 0.9372
INFO:tensorflow:Assets written to: ./saved-models/mini-pipeline-mlp-baseline-model/assets
